In [6]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.regression import LinearRegression, GBTRegressor 
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator


spark = SparkSession.builder \
    .appName("UnemploymentPrediction") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print("Spark Session successfully started.")


Spark Session successfully started.


In [7]:
parquet_path = 'data/integrated_bls_oews_state_year_occ.parquet'
df_spark = spark.read.parquet(parquet_path)

print(f"Total records loaded: {df_spark.count()}")
df_spark.printSchema()
df_spark.show(5, truncate=False)

df_spark = df_spark.withColumn("label", df_spark["State_Unemployment_Rate"].cast("double"))
df_spark = df_spark.drop("State_Unemployment_Rate")

Total records loaded: 312596
root
 |-- Year: long (nullable = true)
 |-- State_FIPS: string (nullable = true)
 |-- Total_Labor_Force: long (nullable = true)
 |-- Total_Employed: long (nullable = true)
 |-- Total_Unemployed: long (nullable = true)
 |-- State_Unemployment_Rate: double (nullable = true)
 |-- OCC_CODE: string (nullable = true)
 |-- OCC_TITLE: string (nullable = true)
 |-- TOT_EMP: double (nullable = true)
 |-- A_MEAN: double (nullable = true)
 |-- A_MEDIAN: double (nullable = true)
 |-- A_PCT10: double (nullable = true)
 |-- A_PCT90: double (nullable = true)

+----+----------+-----------------+--------------+----------------+-----------------------+--------+-----------------------------------+-------+--------+--------+--------+-------+
|Year|State_FIPS|Total_Labor_Force|Total_Employed|Total_Unemployed|State_Unemployment_Rate|OCC_CODE|OCC_TITLE                          |TOT_EMP|A_MEAN  |A_MEDIAN|A_PCT10 |A_PCT90|
+----+----------+-----------------+--------------+-----------

In [8]:
#fill na vals with median
cols_to_fill = ["TOT_EMP", "A_MEAN", "A_MEDIAN", "A_PCT10", "A_PCT90"] 

for col in cols_to_fill:
    try:
        median_value = df_spark.approxQuantile(col, [0.5], 0.01)[0]
        df_spark = df_spark.fillna(median_value, subset=[col])
        print(f"Filled missing values in {col} with median: {median_value:.2f}")
    except Exception as e:
        print(f"Error calculating median for {col}: {e}. Skipping or check data quality.")



categorical_cols = ["State_FIPS", "OCC_CODE"]
indexed_cols = [col + "_Index" for col in categorical_cols]
encoded_cols = [col + "_Vec" for col in categorical_cols]


indexers = [
    StringIndexer(inputCol=col, outputCol=col + "_Index", handleInvalid="keep") 
    for col in categorical_cols
]

# OneHotEncoder
encoders = [
    OneHotEncoder(inputCol=col + "_Index", outputCol=col + "_Vec")
    for col in categorical_cols
]

feature_cols = [
    "Year", "Total_Labor_Force", "Total_Employed", "Total_Unemployed",
    "TOT_EMP", "A_MEAN", "A_MEDIAN", "A_PCT10", "A_PCT90"
] + encoded_cols

for col in feature_cols:
    if col not in encoded_cols:
        df_spark = df_spark.withColumn(col, df_spark[col].cast("double"))

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

# split test_trainiing data
train_data, test_data = df_spark.randomSplit([0.8, 0.2], seed=42)
print(f"\nTraining records: {train_data.count()}, Test records: {test_data.count()}")

Filled missing values in TOT_EMP with median: 830.00
Filled missing values in A_MEAN with median: 50420.00
Filled missing values in A_MEDIAN with median: 47600.00
Filled missing values in A_PCT10 with median: 31750.00
Filled missing values in A_PCT90 with median: 71970.00

Training records: 249954, Test records: 62642


In [9]:
gbt = GBTRegressor(
    featuresCol="features", 
    labelCol="label", 
    maxIter=15,         
    maxDepth=5,          
    seed=42
)


pipeline = Pipeline(stages=indexers + encoders + [assembler, gbt])

print("\n--- Starting Model Training (GBT Regressor) ---")
gbt_model = pipeline.fit(train_data) 

print("Model training complete.")

predictions = gbt_model.transform(test_data)
predictions.select("label", "prediction").show(10)


--- Starting Model Training (GBT Regressor) ---


Model training complete.
+----------------+-----------------+
|           label|       prediction|
+----------------+-----------------+
|6.10014917479779|5.157711044808522|
|6.10014917479779|5.157711044808522|
|6.10014917479779|5.157711044808522|
|6.10014917479779|5.157711044808522|
|6.10014917479779|5.157711044808522|
|6.10014917479779|5.157711044808522|
|6.10014917479779|5.157711044808522|
|6.10014917479779|5.157711044808522|
|6.10014917479779|5.157711044808522|
|6.10014917479779|5.157711044808522|
+----------------+-----------------+
only showing top 10 rows


In [10]:
# evaluate
evaluator_r2 = RegressionEvaluator(
    labelCol="label", 
    predictionCol="prediction", 
    metricName="r2"
)
evaluator_mae = RegressionEvaluator(
    labelCol="label", 
    predictionCol="prediction", 
    metricName="mae"
)

#  R-squared
r2 = evaluator_r2.evaluate(predictions)
print(f"\n--- Model Evaluation Results ---")
print(f"R-squared (R2): {r2:.4f}")

# MAE
mae = evaluator_mae.evaluate(predictions)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

if r2 >= 0.85 and mae <= 0.005: 
    print("\n✅ Model meets project targets!")
else:
    print("\n⚠️ Model still does NOT meet project targets (R2 >= 0.85, MAE <= 0.005).")
    print("Next step: Hyperparameter tuning (CrossValidator) and/or adding ACS PUMS data.")

spark.stop()


--- Model Evaluation Results ---
R-squared (R2): 0.9069
Mean Absolute Error (MAE): 0.3979

⚠️ Model still does NOT meet project targets (R2 >= 0.85, MAE <= 0.005).
Next step: Hyperparameter tuning (CrossValidator) and/or adding ACS PUMS data.
